In [280]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [281]:
df = pd.read_csv('../data_col/FinalDataSetMonths.csv', index_col=0)

# Gestion des dates de sortie
- Remplacement des Year manquantes du dataset original par celles qu'on a scrapées
- Suppression des nan restant dans Year 
- Remplacement des "mauvaises" années de sorties scrapées par l'année du dataset original
- Re-création d'une colonne date_sortie sous format datetime (pour pouvoir ensuite classer par date notre dataset)

In [282]:
df['Year2'] = df['DateSortie'].apply(lambda x: x[:4])
df['Year2'] = df['Year2'].astype(float)
df['Year'] = df['Year'].fillna(df['Year2'])
df = df.drop(['Year2', 'DateSortie'], axis=1)
df['Year'] = df['Year'].replace(0, np.nan)
df = df.dropna(axis = 0, how = 'all', subset = ['Year'])
df['Year'] = df['Year'].astype(int).astype(str)
df['Mois'] = df['Mois'].astype(int).astype(str)
for i,row in df.iterrows():
    if len(df.loc[i, 'Mois']) == 1:
        df.loc[i, 'Date_Sortie'] = df.loc[i,'Year']+'-0'+df.loc[i,'Mois']+'-01'
    else:
        df.loc[i, 'Date_Sortie'] = df.loc[i,'Year']+'-'+df.loc[i,'Mois']+'-01'
df['Date_Sortie'] = pd.to_datetime(df['Date_Sortie'])

In [283]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16538 entries, 0 to 17892
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Name             16538 non-null  object        
 1   Platform         16538 non-null  object        
 2   Year             16538 non-null  object        
 3   Genre            16538 non-null  object        
 4   Publisher        16495 non-null  object        
 5   Studio           13917 non-null  object        
 6   Score            10441 non-null  float64       
 7   Rate             11556 non-null  float64       
 8   Review           9928 non-null   float64       
 9   NA_Sales         16538 non-null  float64       
 10  EU_Sales         16538 non-null  float64       
 11  JP_Sales         16538 non-null  float64       
 12  Other_Sales      16538 non-null  float64       
 13  Global_Sales     16538 non-null  float64       
 14  GK_licence       10863 non-null  objec

# Création des rolling means

## Rolling means par publisher

In [284]:
df_publisher = df.sort_values(by = ['Publisher','Date_Sortie'], ascending=True)
publishers = df_publisher['Publisher'].unique()
publishers = np.delete(publishers, -1)
df_publisher['RM_Publisher'] = np.nan
df_rm1on3 = pd.DataFrame(columns=df_publisher.columns)
for publisher in publishers:
    df_temp = df_publisher[df_publisher['Publisher']==publisher]
    nb_games = df_temp.shape[0]
    if nb_games == 1:
        df_temp.iloc[0, -1] = 0
    elif nb_games == 2:
        df_temp.iloc[0, -1] = 0
        df_temp.iloc[1,-1] = df_temp.iloc[0,13]
    elif nb_games == 3:
        df_temp.iloc[0, -1] = 0
        df_temp.iloc[1,-1] = df_temp.iloc[0,13]
        df_temp.iloc[2,-1] = ((df_temp.iloc[0,13]+df_temp.iloc[1,13])/2)
    else:
        df_temp.iloc[0, -1] = 0
        df_temp.iloc[1,-1] = df_temp.iloc[0,13]
        for i in range(2,nb_games):
            df_temp.iloc[i,-1] = ((df_temp.iloc[i-2,13]+df_temp.iloc[i-1,13])/2)
    df_rm1on3 = pd.concat([df_rm1on3,df_temp], axis = 0)

In [285]:
df_rm1on3

,Name,Platform,Year,Genre,Publisher,Studio,Score,Rate,Review,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,GK_licence,GK_distributeur,Mois,Date_Sortie,RM_Publisher
16962,Pirates: Legend of the Black Buccaneer,PS2,2006,Adventure,10TACLE Studios,WideScreen Games,4.20,7.40,NaN,0.01,0.01,0.00,0.00,0.02,NaN,NaN,8,2006-08-01,0.00
13473,Panzer Tactics,DS,2007,Strategy,10TACLE Studios,Sproing Interactive Media,7.00,9.00,1.0,0.06,0.00,0.00,0.00,0.06,Panzer General,NaN,1,2007-01-01,0.02
15335,Boulder Dash: Rocks!,DS,2007,Puzzle,10TACLE Studios,10tacle Studios,6.50,7.35,7.0,0.00,0.03,0.00,0.00,0.03,Inc.,Electronic Arts,11,2007-11-01,0.04
17739,King's Bounty: Armored Princess,PC,2009,Role-Playing,1C Company,Katauri Interactive,7.85,7.70,33.5,0.00,0.01,0.00,0.00,0.01,"""King's Bounty : The Legend """,Micro Application,11,2009-11-01,0.00
14217,Men of War: Assault Squad,PC,2011,Strategy,1C Company,THQ,7.35,7.75,4.0,0.01,0.03,0.00,0.01,0.05,Men of War,NaN,2,2011-02-01,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17712,SoniPro,3DS,2014,Adventure,imageepoch Inc.,imageepoch,NaN,NaN,NaN,0.00,0.00,0.01,0.00,0.01,Sonicomi,NaN,1,2014-01-01,0.03
11805,Wasteland 2,PC,2015,Role-Playing,inXile Entertainment,InXile Entertainment,7.87,7.25,70.5,0.02,0.06,0.00,0.01,0.10,Wasteland,Deep Silver,9,2015-09-01,0.00
2751,Monster Strike 3DS,3DS,2015,Action,"mixi, Inc",NaN,NaN,NaN,NaN,0.00,0.00,0.86,0.00,0.86,Monster Strike,NaN,1,2015-01-01,0.00
12002,Yourself Fitness,XB,2004,Sports,responDESIGN,Respondesign,8.00,7.00,NaN,0.07,0.02,0.00,0.00,0.09,NaN,NaN,9,2004-09-01,0.00


## Rolling means par studio

In [286]:
df_studio = df.sort_values(by = ['Studio','Date_Sortie'], ascending=True)
studios = df_studio['Studio'].unique()
studios = np.delete(studios, -1)
df_studio['RM_Studio'] = np.nan
df_rm2on3 = pd.DataFrame(columns=df_studio.columns)
for studio in studios:
    df_temp = df_studio[df_studio['Studio']==studio]
    nb_games = df_temp.shape[0]
    if nb_games == 1:
        df_temp.iloc[0, -1] = 0
    elif nb_games == 2:
        df_temp.iloc[0, -1] = 0
        df_temp.iloc[1,-1] = df_temp.iloc[0,13]
    elif nb_games == 3:
        df_temp.iloc[0, -1] = 0
        df_temp.iloc[1,-1] = df_temp.iloc[0,13]
        df_temp.iloc[2,-1] = ((df_temp.iloc[0,13]+df_temp.iloc[1,13])/2)
    else:
        df_temp.iloc[0, -1] = 0
        df_temp.iloc[1,-1] = df_temp.iloc[0,13]
        for i in range(2,nb_games):
            df_temp.iloc[i,-1] = ((df_temp.iloc[i-2,13]+df_temp.iloc[i-1,13])/2)
    df_rm2on3 = pd.concat([df_rm2on3,df_temp], axis = 0)

In [287]:
df_rm2on3

,Name,Platform,Year,Genre,Publisher,Studio,Score,Rate,Review,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,GK_licence,GK_distributeur,Mois,Date_Sortie,RM_Studio
9556,Eve: The Lost One,SAT,1998,Adventure,Imagineer,"""C's Ware""",NaN,7.00,1.0,0.00,0.00,0.16,0.00,0.16,Eve,NaN,1,1998-01-01,0.000
6955,F-1 Grand Prix,SNES,1992,Racing,Video System,"""Traveller's Tales""",3.00,NaN,NaN,0.00,0.00,0.28,0.00,0.28,Formula One (Psygnosis/Sony),NaN,1,1992-01-01,0.000
5158,Rascal,PS,1998,Action,Psygnosis,"""Traveller's Tales""",NaN,6.00,1.0,0.24,0.16,0.00,0.03,0.42,NaN,NaN,1,1998-01-01,0.280
9643,Muppet RaceMania,PS,1999,Racing,Sony Computer Entertainment,"""Traveller's Tales""",NaN,NaN,NaN,0.09,0.06,0.00,0.01,0.15,Muppet,NaN,1,1999-01-01,0.350
7507,WRC: World Rally Championship,PS2,2001,Racing,Sony Computer Entertainment,"""Traveller's Tales""",5.00,5.50,2.0,0.12,0.09,0.00,0.03,0.25,World Rally Championship,NaN,1,2001-01-01,0.285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,Digimon World Re:Digitize,PSP,2012,Action,Namco Bandai Games,tri-Crescendo,NaN,7.00,1.0,0.00,0.00,0.22,0.00,0.22,Digimon,NaN,1,2012-01-01,0.000
12215,Digimon World Re:Digitize Decode,3DS,2013,Role-Playing,Namco Bandai Games,tri-Crescendo,NaN,NaN,NaN,0.00,0.00,0.09,0.00,0.09,Digimon,NaN,1,2013-01-01,0.220
15668,Hot Pixel,PSP,2007,Puzzle,Atari,zSlide,5.25,7.25,2.0,0.03,0.00,0.00,0.00,0.03,NaN,NaN,10,2007-10-01,0.000
17770,Muv-Luv Alternative,PS3,2012,Simulation,5pb,ǽge-soft,NaN,8.00,3.0,0.00,0.00,0.01,0.00,0.01,Muv-luv,NaN,1,2012-01-01,0.000


## Rolling means par licence

In [288]:
df_licence = df.sort_values(by = ['GK_licence','Date_Sortie'], ascending=True)
licences = df_licence['GK_licence'].unique()
licences = np.delete(licences, -1)
df_licence['RM_Licence'] = np.nan
df_rm3on3 = pd.DataFrame(columns=df_licence.columns)
for licence in licences:
    df_temp = df_licence[df_licence['GK_licence']==licence]
    nb_games = df_temp.shape[0]
    if nb_games == 1:
        df_temp.iloc[0, -1] = 0
    elif nb_games == 2:
        df_temp.iloc[0, -1] = 0
        df_temp.iloc[1,-1] = df_temp.iloc[0,13]
    elif nb_games == 3:
        df_temp.iloc[0, -1] = 0
        df_temp.iloc[1,-1] = df_temp.iloc[0,13]
        df_temp.iloc[2,-1] = ((df_temp.iloc[0,13]+df_temp.iloc[1,13])/2)
    else:
        df_temp.iloc[0, -1] = 0
        df_temp.iloc[1,-1] = df_temp.iloc[0,13]
        for i in range(2,nb_games):
            df_temp.iloc[i,-1] = ((df_temp.iloc[i-2,13]+df_temp.iloc[i-1,13])/2)
    df_rm3on3 = pd.concat([df_rm3on3,df_temp], axis = 0)

In [289]:
df_rm3on3

,Name,Platform,Year,Genre,Publisher,Studio,Score,Rate,Review,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,GK_licence,GK_distributeur,Mois,Date_Sortie,RM_Licence
10091,Akiba's Trip,PSP,2011,Adventure,Acquire,NaN,3.0,5.70,19.0,0.00,0.00,0.14,0.00,0.14,"""Akiba's Trip """,NaN,1,2011-01-01,0.00
12593,Akiba's Trip Plus,PSP,2012,Adventure,Acquire,Acquire,NaN,NaN,NaN,0.00,0.00,0.08,0.00,0.08,"""Akiba's Trip """,NaN,1,2012-01-01,0.14
7498,America's Army: Rise of a Soldier,XB,2005,Shooter,Ubisoft,Secret Level,7.0,7.40,NaN,0.19,0.05,0.00,0.01,0.25,"""America's Army """,NaN,11,2005-11-01,0.00
12347,America's Army: True Soldiers,X360,2007,Shooter,Ubisoft,Red Storm Entertainment,4.3,6.70,NaN,0.08,0.00,0.00,0.01,0.08,"""America's Army """,NaN,11,2007-11-01,0.25
7316,A.C.E.: Another Century's Episode,PS2,2005,Simulation,Banpresto,FromSoftware,NaN,9.00,1.0,0.00,0.00,0.26,0.00,0.26,"""Another Century's Episode """,NaN,1,2005-01-01,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3716,Toshochu: Run for Money,3DS,2012,Adventure,Namco Bandai Games,NaN,NaN,NaN,NaN,0.00,0.00,0.62,0.00,0.62,run for money,NaN,1,2012-01-01,0.00
17252,Super Battle For Money Sentouchuu: Kyuukyoku n...,3DS,2016,Action,Namco Bandai Games,NaN,NaN,NaN,NaN,0.00,0.00,0.01,0.00,0.01,run for money,NaN,1,2016-01-01,0.62
8697,uDraw Studio: Instant Artist,Wii,2011,Misc,THQ,THQ,NaN,5.25,6.0,0.08,0.09,0.00,0.02,0.19,uDraw Studio,NaN,11,2011-11-01,0.00
16774,uDraw Studio: Instant Artist,X360,2011,Misc,THQ,THQ,5.4,4.45,6.0,0.01,0.01,0.00,0.00,0.02,uDraw Studio,NaN,11,2011-11-01,0.19


## Merge des datasets

In [290]:
df_rm = df.merge(right=df_rm1on3, on=['Name', 'Platform', 'Year', 'Genre', 'Publisher', 'Studio', 'Score',
       'Rate', 'Review', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales',
       'Global_Sales', 'GK_licence', 'GK_distributeur', 'Mois', 'Date_Sortie'], how='left')

In [291]:
df_rm = df_rm.merge(right=df_rm2on3, on=['Name', 'Platform', 'Year', 'Genre', 'Publisher', 'Studio', 'Score',
       'Rate', 'Review', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales',
       'Global_Sales', 'GK_licence', 'GK_distributeur', 'Mois', 'Date_Sortie'], how='left')

In [292]:
df_rm = df_rm.merge(right=df_rm3on3, on=['Name', 'Platform', 'Year', 'Genre', 'Publisher', 'Studio', 'Score',
       'Rate', 'Review', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales',
       'Global_Sales', 'GK_licence', 'GK_distributeur', 'Mois', 'Date_Sortie'], how='left')

In [293]:
df_rm.isna().sum()

Name                   0
Platform               0
Year                   0
Genre                  0
Publisher             43
Studio              2621
Score               6097
Rate                4982
Review              6610
NA_Sales               0
EU_Sales               0
JP_Sales               0
Other_Sales            0
Global_Sales           0
GK_licence          5675
GK_distributeur    14123
Mois                   0
Date_Sortie            0
RM_Publisher          43
RM_Studio           2621
RM_Licence          5675
dtype: int64

# Gestion des autres NaN

## Suppression de GK_distributeur

In [294]:
df_rm = df_rm.drop(['GK_distributeur'], axis=1)

In [295]:
df_rm.isna().sum()

Name               0
Platform           0
Year               0
Genre              0
Publisher         43
Studio          2621
Score           6097
Rate            4982
Review          6610
NA_Sales           0
EU_Sales           0
JP_Sales           0
Other_Sales        0
Global_Sales       0
GK_licence      5675
Mois               0
Date_Sortie        0
RM_Publisher      43
RM_Studio       2621
RM_Licence      5675
dtype: int64

## Remplacement des NaN pour score et rate

In [296]:
df_rm['Score'] = df_rm['Score'].fillna(df_rm['Rate'])
df_rm['Rate'] = df_rm['Rate'].fillna(df_rm['Score'])
df_rm.isna().sum()

Name               0
Platform           0
Year               0
Genre              0
Publisher         43
Studio          2621
Score           4203
Rate            4203
Review          6610
NA_Sales           0
EU_Sales           0
JP_Sales           0
Other_Sales        0
Global_Sales       0
GK_licence      5675
Mois               0
Date_Sortie        0
RM_Publisher      43
RM_Studio       2621
RM_Licence      5675
dtype: int64

## dropna global

In [297]:
df_rm = df_rm.dropna()
df_rm

,Name,Platform,Year,Genre,Publisher,Studio,Score,Rate,Review,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,GK_licence,Mois,Date_Sortie,RM_Publisher,RM_Studio,RM_Licence
0,Wii Sports,Wii,2006,Sports,Nintendo,Nintendo,7.6,7.05,210.0,41.49,29.02,3.77,8.46,82.74,Wii Sports,11,2006-11-01,2.840,15.605,0.000
1,Super Mario Bros.,NES,1985,Platform,Nintendo,Nintendo,7.0,8.40,235.0,29.08,3.58,6.81,0.77,40.24,Mario,1,1985-01-01,1.045,1.495,1.935
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,Nintendo,7.6,8.00,319.0,15.85,12.88,3.79,3.31,35.82,Mario,4,2008-04-01,0.380,0.335,11.030
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,Nintendo,7.5,7.35,103.0,15.75,11.01,3.28,2.96,33.00,Wii Sports,7,2009-07-01,0.565,6.250,82.740
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,Nintendo,7.0,8.20,138.0,11.27,8.89,10.22,1.00,31.37,PokǸmon,1,1996-01-01,2.620,2.605,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16508,XI Coliseum,PSP,2006,Puzzle,Sony Computer Entertainment,SNK Playmore,7.0,7.90,35.0,0.00,0.00,0.01,0.00,0.01,The King of Fighters,1,2006-01-01,0.015,0.175,0.030
16509,Resident Evil 4 HD,XOne,2016,Shooter,Capcom,Capcom,9.0,8.70,624.0,0.01,0.00,0.00,0.00,0.01,Resident Evil,1,2016-01-01,0.010,0.015,0.015
16510,Farming 2017 - The Simulation,PS4,2016,Simulation,UIG Entertainment,Giants Software,6.5,6.50,2.0,0.00,0.01,0.00,0.00,0.01,Farming Simulator,1,2016-01-01,0.000,0.010,0.015
16523,Carmageddon 64,N64,1999,Action,Virgin Interactive,Software Creations,5.0,5.00,1.0,0.01,0.00,0.00,0.00,0.01,Carmageddon,1,1999-01-01,0.020,1.060,0.000


In [298]:
df_rm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7133 entries, 0 to 16536
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Name          7133 non-null   object        
 1   Platform      7133 non-null   object        
 2   Year          7133 non-null   object        
 3   Genre         7133 non-null   object        
 4   Publisher     7133 non-null   object        
 5   Studio        7133 non-null   object        
 6   Score         7133 non-null   float64       
 7   Rate          7133 non-null   float64       
 8   Review        7133 non-null   float64       
 9   NA_Sales      7133 non-null   float64       
 10  EU_Sales      7133 non-null   float64       
 11  JP_Sales      7133 non-null   float64       
 12  Other_Sales   7133 non-null   float64       
 13  Global_Sales  7133 non-null   float64       
 14  GK_licence    7133 non-null   object        
 15  Mois          7133 non-null   object 

# Machine Learning

In [299]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

# Modèle de classification que l'on va utiliser
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Pour évaluer nos modèles
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc

In [300]:
df_rm['Sales_cat'] = pd.cut(df_rm['Global_Sales'], bins=[0,0.1,0.249,1, 100000], 
       labels=['01_Moins de 0.1M', '02_Entre 0.1M et 0.25M', '03_Entre 0.25M et 1M', '04_Plus de 1M'])
df_rm = df_rm.drop(['Name', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'], axis=1)

cat_cols = df_rm.select_dtypes(include=['object']).columns.to_list()
for col in cat_cols:
    i = 1
    occurences = df_rm[col].unique()
    for occ in occurences:
        df_rm.loc[df_rm[col]==occ, col] = i
        i+=1
        
df_rm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7133 entries, 0 to 16536
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Platform      7133 non-null   object        
 1   Year          7133 non-null   object        
 2   Genre         7133 non-null   object        
 3   Publisher     7133 non-null   object        
 4   Studio        7133 non-null   object        
 5   Score         7133 non-null   float64       
 6   Rate          7133 non-null   float64       
 7   Review        7133 non-null   float64       
 8   GK_licence    7133 non-null   object        
 9   Mois          7133 non-null   object        
 10  Date_Sortie   7133 non-null   datetime64[ns]
 11  RM_Publisher  7133 non-null   float64       
 12  RM_Studio     7133 non-null   float64       
 13  RM_Licence    7133 non-null   float64       
 14  Sales_cat     7133 non-null   category      
dtypes: category(1), datetime64[ns](1), fl

In [301]:
df_rm[cat_cols] = df_rm[cat_cols].astype('float')
df_rm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7133 entries, 0 to 16536
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Platform      7133 non-null   float64       
 1   Year          7133 non-null   float64       
 2   Genre         7133 non-null   float64       
 3   Publisher     7133 non-null   float64       
 4   Studio        7133 non-null   float64       
 5   Score         7133 non-null   float64       
 6   Rate          7133 non-null   float64       
 7   Review        7133 non-null   float64       
 8   GK_licence    7133 non-null   float64       
 9   Mois          7133 non-null   float64       
 10  Date_Sortie   7133 non-null   datetime64[ns]
 11  RM_Publisher  7133 non-null   float64       
 12  RM_Studio     7133 non-null   float64       
 13  RM_Licence    7133 non-null   float64       
 14  Sales_cat     7133 non-null   category      
dtypes: category(1), datetime64[ns](1), fl

In [305]:
target = df_rm['Sales_cat']
data = df_rm.drop(['Sales_cat', 'Date_Sortie'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

In [306]:
# Définition du modèle
lr = LogisticRegression()
knn = KNeighborsClassifier()
rf = RandomForestClassifier()
dt = DecisionTreeClassifier()
svm = SVC()

# Entraînement des modèles
lr.fit(X_train, y_train)
print("score du lr : {}".format(lr.score(X_test, y_test)))

knn.fit(X_train, y_train)
print("score du knn : {}".format(knn.score(X_test, y_test)))

rf.fit(X_train, y_train)
print("score du rf : {}".format(rf.score(X_test, y_test)))

dt.fit(X_train, y_train)
print("score du dt : {}".format(dt.score(X_test, y_test)))

svm.fit(X_train, y_train)
print("score du svm : {}".format(svm.score(X_test, y_test)))

score du lr : 0.524176594253679
score du knn : 0.5108619481429573
score du rf : 0.6986685353889278
score du dt : 0.5844428871758934
score du svm : 0.5732305536089699


In [307]:
param_grid = [{
            'n_estimators': [10, 50, 100, 250, 500, 1000],
            'min_samples_leaf': [1, 3, 5],
            'max_features': ['sqrt', 'log2']
}]

In [308]:
CV_rfc = GridSearchCV(estimator=rf, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)
CV_rfc.best_params_

{'max_features': 'log2', 'min_samples_leaf': 1, 'n_estimators': 1000}

In [309]:
rfc1=RandomForestClassifier(
    max_features=CV_rfc.best_params_['max_features'], 
    n_estimators=CV_rfc.best_params_['n_estimators'], 
    min_samples_leaf=CV_rfc.best_params_['min_samples_leaf'])
rfc1.fit(X_train, y_train)
print("score du best rf : {}".format(rfc1.score(X_test, y_test)))

score du best rf : 0.7021723896285914


In [310]:
y_pred = rfc1.predict(X_test)
cm = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
cm

Classe prédite,01_Moins de 0.1M,02_Entre 0.1M et 0.25M,03_Entre 0.25M et 1M,04_Plus de 1M
Classe réelle,,,,
01_Moins de 0.1M,230,53,46,12
02_Entre 0.1M et 0.25M,36,151,83,2
03_Entre 0.25M et 1M,25,44,409,62
04_Plus de 1M,7,1,54,212
